# Local Search: Basics

---

**Local search** is a very commonly used heuristic for solving complex combinatorial optimisation problems. In this tutorial, we will introduce the basics of local search.

## Local Search Framework

---

The idea of local search is very simple. It starts with an **initial solution**. At each step, it explores the **neighborhood** of the current solution, and **move to a neighbouring solution**.

<img src="img/ls.png" width=500 />

The framework of the local search is as follows.

```Python
def local_search_framework(problem):
    sol = initialise(problem)
    while not stop:
        neighbour = select_from_neighbourhood(sol, problem)
    
        if accept(neighbour, sol, problem):
            sol = neighbour
            
    return sol
```

The local search has the following design issues:

1. How to **initialise** the solution?
2. How to **define the neighbourhood**?
3. How to **select the neighbour** from the neighbourhood of the current solution?
4. How to decide whether to **accept the selected neighbour** (move to the neighbour) or not?
5. How to define the **stopping criteria**?

## Solution Initialisation

---

The solution initialisation is problem specific. 

The most straightforward way is to **randomly generate** a solution. For example, for the traveling salesman problem where a solution is a permutation of the given set of nodes, we can randomly generate a solution by randomly shuffling the nodes. For the knapsack problem to select a subset of items subject to the capacity constraint, we can randomly pick the items until there is no item to be added without violating the capacity constraint.

We could also use **constructive heuristics** to initialise the solution. Examples include the [nearest neighbour and insertion heuristics for traveling salesman problem](https://github.com/meiyi1986/tutorials/blob/master/notebooks/tsp-greedy.ipynb), and [efficiency first heuristic for knapsack problem](https://github.com/meiyi1986/tutorials/blob/master/notebooks/knapsack-greedy.ipynb). The constructive heuristics can give much better initial solutions than random generation. However, they might also be too greedy and restrict the search within a narrow region around it.

## Neighbourhood Definition

---

The neighbourhood definition is also problem specific.

In **continuous optimisation** where the decision variables $\mathbf{x}$ are continuous numbers, the neighbourhood definition is straightforward and intuitive. We can say that the neighbourhood of a solution $\mathbf{x}$ is the region (set of solutions) within a certain radius around it, i.e., 

$$
\mathcal{N}_{\delta}(\mathbf{x}) = \{\mathbf{x}' \text{ with } ||\mathbf{x}' - \mathbf{x}|| < \delta\},
$$

where $\delta$ is the radius parameter.

However, in **combinatorial optimisation**, the neighbourhood structure is not straightforward due to the discontinuous, non-smooth and irregular structure of solutions, such as a permutation (routing and scheduling) or binary string (knapsack problem).

### Move Operators

In general, the neighbourhood of combinatorial optimisation problems is defined based on **move operators**, which is an operator to (slightly) modify the solution. Specifically, given a solution $x$, a move operator $\mathtt{op}$ and its associated parameters $\boldsymbol{\theta}$, we can obtain a new solution

$$
x' = \mathtt{op}(x, \boldsymbol{\theta}).
$$

Then, the **neighbourhood** based on the move operator $\mathtt{op}$ is defined as the set of $x' = \mathtt{op}(x, \boldsymbol{\theta})$ for all the possible $\boldsymbol{\theta}$ values, i.e.,

$$
\mathcal{N}_{\mathtt{op}}(x) = \bigcup_{\boldsymbol{\theta} \in \boldsymbol{\Theta}} \mathtt{op}(x, \boldsymbol{\theta}),
$$

where $\boldsymbol{\Theta}$ is the set of all the possible $\boldsymbol{\theta}$ values.

Below are some examples of commonly used move operators for combinatorial optimisation problems:

- **2-opt** for traveling salesman problem: Given a tour, which is a permutation of nodes $[x_1, \dots, x_n, x_1]$, the 2-opt operator selects a sub-tour $[x_i, \dots, x_j]$, and reverse it. The resultant solution is $[x_1, \dots, x_{i-1}, x_j, x_{j-1}, \dots, x_{i}, x_{j+1}, \dots, x_1]$. The associated parameters are $i$ and $j$.
- **Bit flip** for binary optimisation such as knapsack problem: Given a bit string, the bit flip operator flips the bit (1 to 0, 0 to 1) of a position $i$. The associated parameter is $i$.

> **NOTE**: **The design of move operators is the key to the success of local search.** However, designing effective move operators heavily relies on domain knowledge and expertise.

> **NOTE**: The neighbourhood of combinatorial optimisation generally consists of a **finite number** of neighbours, although the number can be large. In contrast, the continuous optimisation typically has an infinite neighbours.

### Local Optimum

Based on the defined neighbourhood, the concept of **local optimum** is defined as follows.

> **DEFINITION**: A solution $x^*$ is a local optimum in the neighbourhood $\mathcal{N}(\cdot)$, if there is no neighbour $x' \in \mathcal{N}(x^*)$ that is better than $x^*$.

Note that whether a solution $x$ is a local optimum or not depends on the neighbourhood $\mathcal{N}(\cdot)$, which determines the set of neighbours. A solution might be a local optimum in one neighbourhood $\mathcal{N}_1(\cdot)$, but not a local optimum in another neighbourhood $\mathcal{N}_2(\cdot)$.

## Neighbour Selection

---

Below are some commonly used neighbour selection schemes.

- Randomly sample a neighbour from the neighbourhood.
- Select the first neighbour that is better than the current solution (**first improvement**).
- Select the best neighbour in the neighbourhood (**best improvement**).

Different local search algorithms might use different neighbour selection schemes.

## Neighbour Acceptance

---

The common neighbour acceptance criteria are as follows.

- Accept the neighbour if it is better than the current solution.
- Always accept the neighbour.
- Accept based on probability.

## Stopping Criteria

---

There are several possible stopping criteria that can be used.

- When no improvement is found in the current neighbourhood (the search is stuck in a local optimum).
- When a certain number of iterations is reached.

## Common Local Search Algorithms

---

Now, we introduce some common local search algorithms and how they fit into the local search framework and address the design issues in different ways. In general, the solution initialisation and neighbourhood definition are problem specific, and shared by different local search algorithms. Here, we focus on **Neighbour Selection**, **Neighbour Acceptance** and **Stopping Criteria**.

> **NOTE**: Without loss of generality, we assume minimisation problem $\min f(x)$.

### Hill Climbing

[Hill Climbing](https://en.wikipedia.org/wiki/Hill_climbing) is the most basic local search algorithm. It keeps moving to the best neighbour until reaching a local optimum.

#### Neighbour Selection

At each step, the **best neighbour** in the neighbourhood of the current solution is selected, i.e., 

```Python
def select_from_neighbourhood(sol, problem):
    return argmin([f(nb) for nb in neighbourhood(sol)])
```

#### Neighbour Acceptance

The neighbour is **accepted if it is better** than the current solution, i.e., 

```Python
def accept(nb, sol, problem):
    return f(nb) < f(sol)
```

#### Stopping Criteria

The search is stopped after reaching a local optimum. That is, `obj(nb) >= obj(sol)`.

### Simulated Annealing

[Simulated Annealing](https://en.wikipedia.org/wiki/Simulated_annealing) is inspired by the metal annealing process. It might jump out of local optima with a probability controlled by a temperature parameter.

#### Neighbour Selection

At each step, a neighbour is **randomly** sampled from the neighbourhood of the current solution, i.e., 

```Python
def select_from_neighbourhood(sol, problem):
    randomly sample a neighbour from neighbourhood(sol)
```

#### Neighbour Acceptance

A neighbour is **always accepted if it is better** than the current solution. **Otherwise it is accepted with probability** $e^{\frac{f(x)-f(x')}{T}}$, where $T \geq 0$ is the temperature.

```Python
def accept(nb, sol, problem):
    if f(nb) < f(sol):
        return True
    elif random.rand() < exp((f(sol) - f(nb)) / T):
        return True
    return False
```

#### Stopping Criteria

The search is stopped after a certain number of iterations.

### Tabu Search

[Tabu Search](https://en.wikipedia.org/wiki/Tabu_search) employs a tabu list to store the search memory. The search is guided by the tabu list not to go back to previously visited solutions, so as to escape from local optima.

#### Neighbour Selection

At each step, the **best non-tabu** neighbour in the neighbourhood of the current solution is selected, i.e.,

```Python
def select_from_neighbourhood(sol, problem):
    argmin([f(nb) for nb in neighbourhood(sol) if nb not in tabu_list])
```

#### Neighbour Acceptance

A neighbour is **always accepted** no matter whether it is better than the current solution or not, i.e.,

```Python
def accept(nb, sol, problem):
    return True
```

#### Stopping Criteria

The search is stopped after a certain number of iterations.

### Guided Local Search

[Guided Local Search](https://en.wikipedia.org/wiki/Guided_Local_Search) employs a utility function to penalise each feature of solutions, and accept neighbours based on the **augmented** objective function by the utility instead of the original objective function. By penalisation, the local optimum's augmented objective function becomes worse and worse, and the search can jump to other neighbours.

#### Neighbour Selection

At each step, the neighbour in the neighbourhood of the current solution with the best augmented function is selected, i.e.,

```Python
def select_from_neighbourhood(sol, problem):
    argmin([g(nb) for nb in neighbourhood(sol)])
```

where `g(nb)` is the augmented objective function with the utility penalisation.

#### Neighbour Acceptance

A neighbour is **accepted if it has better augmented objective function** than the current solution, i.e.,

```Python
def accept(nb, sol, problem):
    return g(nb) < g(sol)
```

#### Stopping Criteria

The search is stopped after a certain number of iterations.

## Jump Out of Local Optima

---

Local search is easy to be trapped into local optima, especially for complex multi-modal problems with many local optima. Therefore, it is important to design mechanisms for local search to **jump out of local optima**.

Below are some commonly used strategies to jump out of local optima. 

### Accept Worse Neighbours

We can accept worse neighbours to help the search move out of the current valley and find another (better) local optimum. Example algorithms include simulated annealing (accept with some probability) and tabu search (accept non-tabu worse neighbours).

### Change Neighbourhood

Noted that local optimum depends on the neighbourhood. A local optimum in one neighbourhood might become a non-local optimum in another neighbourhood. Thus, we can change the neighbourhood (usually to a larger neighbourhood) to make the current local optimum a non-local optimum, so the search can jump out of the current local optimum. An example algorithm is the [variable neighbourhood search](https://en.wikipedia.org/wiki/Variable_neighborhood_search).

### Change Evaluation

We can also change the evaluation scheme to consider additional aspects to the original objective function. For example, we can penalise solutions whose building blocks have been explored too much, so that we can explore new solutions that have not been seen before. An example algorithm is the guided local search.

### Restart

After the search gets stuck, we can restart from a new initial solution and do the local search again. If the new initial solution is substantially different from the current local optimum (and the solutions examined before), then we are likely to find a new (possibly better) local optimum.

The new initial solution could be obtained randomly or perturbing the current local optimum. This is the idea of the [iterated local search](https://en.wikipedia.org/wiki/Iterated_local_search).

---

- More tutorials can be found [here](https://github.com/meiyi1986/tutorials).
- [Yi Mei's homepage](https://meiyi1986.github.io/)